# Importación Librerías

In [8]:
import pandas as pd
from io import StringIO
import gdown
import json
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ast
from collections import Counter



# Reviews Estados - Google

## Importar archivos (cambiar a la nube)

In [22]:
# Leer el archivo y almacenar cada línea en una lista
lines = []

for i in range (18):
    file='reviews-estados/review-California/'+str(i+1)+'.json'
    with open(file, 'r') as file:
        for line in file:
            lines.append(line)

In [23]:
# Unir todas las líneas en un solo string y leerlo como un DataFrame
all_lines = ''.join(lines)
df_reviews = pd.read_json(StringIO(all_lines), lines=True)

In [24]:
del lines
del all_lines
del file
del line

Espacio que ocupan
Carpeta: 763 Mb (3,8 segundos en importar para una lista, 48,44 segundos para pasarlo a dataframe)
Dataframe: 164,8 Mb
CSV: 545 Mbs

# Metadata  - Google

## Importar los archivos desde la nube

tamaño:
carpeta: 2,76 Gb
dataframe 370 Mb
csv 2,3 Gb

In [16]:
## descargados directamente desde google drive

# IDs de los archivos de Google Drive
file_links = [
    'https://drive.google.com/file/d/1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS', 
    'https://drive.google.com/file/d/15D_5EkxqPP0XJb5I5bYI8b1wQV7B2fx_', 
    'https://drive.google.com/file/d/1fDBVCmf4JA7gkIyjpv5mHEMySb19C-vz', 
    'https://drive.google.com/file/d/1Mj2oUZy5gGznhthcUGi8_sgKhBwypE74', 
    'https://drive.google.com/file/d/1IXok40Zp61CGwFDgyvLUwV02c4BWGrjj',
    'https://drive.google.com/file/d/1UmsN_ZOFQqVl7W9SbnxHkSQavo1_Iwqx', 
    'https://drive.google.com/file/d/1KfQBhJlnuziKjf-9haQGaiPtCPnUUDla', 
    'https://drive.google.com/file/d/1ebTUx2klGy7L9lGlZl3GCPXxSwSD55vX', 
    'https://drive.google.com/file/d/1td6twU60LAS-z5mB0MeSJEpGhH7jcGKm', 
    'https://drive.google.com/file/d/1NQgHgNm9PV8MSiOXNoQ2UkIF9e5AdLk7', 
    'https://drive.google.com/file/d/1GYwWfH7EvWMZn14vQRNr5CjEely4eWrB'
]

# Nombres de los archivos locales (presumiendo que siguen el patrón 1.json, 2.json, ..., 11.json)
file_names = [f'{i}.json' for i in range(1, len(file_links) + 1)]

# Inicializar una lista para almacenar los DataFrames
dataframes = []

# Descargar y leer cada archivo JSON
for file_link, file_name in zip(file_links, file_names):
    try:
        # Obtener el ID del archivo desde el enlace
        file_id = file_link.split('/d/')[1].split('/')[0]
        download_url = f'https://drive.google.com/uc?id={file_id}'
        
        # Descargar el archivo
        gdown.download(download_url, file_name, quiet=False)
        
        # Verificar si el archivo descargado es un JSON válido
        with open(file_name, 'r', encoding='utf-8') as file:
            first_char = file.read(1)
            if not first_char:
                raise ValueError(f"El archivo {file_name} está vacío.")
            file.seek(0)
        
        # Leer el archivo JSON en un DataFrame
        df = pd.read_json(file_name, lines=True)
        
        # Agregar el DataFrame a la lista
        dataframes.append(df)
        
        # Eliminar el archivo descargado
        os.remove(file_name)
    
    except Exception as e:
        print(f"Error al procesar el archivo {file_name}: {e}")
        if os.path.exists(file_name):
            os.remove(file_name)

Downloading...
From (original): https://drive.google.com/uc?id=1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS
From (redirected): https://drive.google.com/uc?id=1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS&confirm=t&uuid=03f8792e-1011-4b2d-8196-607d15726cee
To: c:\Users\feder\Desktop\PF\1.json
100%|██████████| 256M/256M [00:06<00:00, 37.7MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=15D_5EkxqPP0XJb5I5bYI8b1wQV7B2fx_
From (redirected): https://drive.google.com/uc?id=15D_5EkxqPP0XJb5I5bYI8b1wQV7B2fx_&confirm=t&uuid=ab4b6a9d-8aee-4153-9cc4-73dc4064ccae
To: c:\Users\feder\Desktop\PF\2.json
100%|██████████| 257M/257M [00:09<00:00, 27.3MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1fDBVCmf4JA7gkIyjpv5mHEMySb19C-vz
From (redirected): https://drive.google.com/uc?id=1fDBVCmf4JA7gkIyjpv5mHEMySb19C-vz&confirm=t&uuid=d854ef6a-e45c-47f1-b98a-9a99e26300d0
To: c:\Users\feder\Desktop\PF\3.json
100%|██████████| 259M/259M [00:07<00:00, 33.1MB/s] 
Downloading...
From (original): ht

In [17]:
del df

In [18]:
# Concatenar todos los DataFrames en uno solo
df_metadataGoog = pd.concat(dataframes, ignore_index=True)

In [21]:
del dataframes

## Filtrar por estado california

In [22]:
#extraer el estado de la dirección
df_metadataGoog['estado'] = df_metadataGoog['address'].str.extract(r', ([A-Z]{2}) \d{5}')

In [23]:
#filtro los datos de California, para liberar espacio
df_metadatosCA = df_metadataGoog[df_metadataGoog['estado'] == 'CA']

In [24]:
#elimino el dataframe que tiene los metadatos de todos los estados, para liberar memoria
del df_metadataGoog

In [25]:
df_metadatosCA.reset_index(inplace=True)

In [26]:
df_metadatosCA.drop('index', axis='columns', inplace=True)

C:\Users\feder\AppData\Local\Temp\ipykernel_7120\1024337664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadatosCA.drop('index', axis='columns', inplace=True)


## Exportar el archivo

In [28]:
#guardo el archivo parquet, para poder importarlo si es necesario
df_metadatosCA.to_parquet('Archivos/metadatosCA.parquet', engine='pyarrow')

# Business - YELP

## Importar los archivos desde la nube

In [10]:
# ID del archivo de Google Drive
file_id = '1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu'
# Construir la URL de descarga
download_url = f'https://drive.google.com/uc?id={file_id}'

# Descargar el archivo usando gdown
file_path = 'business.pkl'
gdown.download(download_url, file_path, quiet=False)

# Cargar el archivo pickle en un DataFrame de pandas
df_businessYelp = pd.read_pickle(file_path)

# Eliminar el archivo descargado
os.remove(file_path)

Downloading...
From (original): https://drive.google.com/uc?id=1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu
From (redirected): https://drive.google.com/uc?id=1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu&confirm=t&uuid=69cf883e-c12b-4736-b320-965493d099b8
To: c:\Users\feder\Desktop\PF\business.pkl
100%|██████████| 116M/116M [00:03<00:00, 36.9MB/s] 


#  Business - YELP
### Importación de los Datos 

In [3]:
# Ruta del archivo .pkl
ruta_pkl = 'D:/usuarios/CIN/MIS DOCUMENTOS/Proyecto_Final/set_de_datos_Yelp/business.pkl'

# Cargar el archivo .pkl en un DataFrame de pandas
df_business = pd.read_pickle(ruta_pkl)

### Transformación de los Datos 


In [5]:
# Elimina las columnas duplicadas
df = df_business.loc[:, ~df_business.columns.duplicated()]

#  Filtra negocios en un estado específico (por ejemplo, 'CA' para California)
business_california = df[df['state'] == 'CA']

# Elimina valores nulos en las columnas attributes y hours
business_california_sn= business_california.dropna(subset=['attributes', 'hours'])

# Modificación del tipo  de datos en las columnas 'latitude','longitud','stars','review_count','is-open'

business_california_sn['latitude'] = pd.to_numeric(business_california_sn['latitude'], errors='coerce')
business_california_sn['longitude'] = pd.to_numeric(business_california_sn['longitude'], errors='coerce')
business_california_sn['stars'] = pd.to_numeric(business_california_sn['stars'], errors='coerce')
business_california_sn['review_count'] = pd.to_numeric(business_california_sn['review_count'], errors='coerce')
business_california_sn['is_open'] = pd.to_numeric(business_california_sn['is_open'], errors='coerce')

In [14]:
#guardo el archivo parquet, para poder importarlo si es necesario
business_california_sn.to_parquet('Archivos/business_CA_Yelp', engine='pyarrow')

# Review - YELP
## Importación de los Datos 
### para realiza la importacion de los datos de reviews se leyó el archivo por Chunks o bloques de 1000 lineas , luego se hizo un filtrado por el business_id , correspondiente 

In [ ]:


# Definir la ruta al archivo JSON
file_path = '/content/drive/My Drive/set_de_datos_Yelp/review (1).json'

# Definir el tamaño del chunk (número de líneas por chunk)
chunk_size = 1000  # Puedes ajustar este valor según sea necesario

# Lista para almacenar los DataFrames procesados
results = []

# Leer el archivo JSON en chunks
chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)

# Procesar cada chunk por separado
for i, chunk in enumerate(chunks):
    # Procesar el chunk como sea necesario
    print(f"Procesando chunk {i + 1}")

    # Ejemplo de procesamiento: mostrar las primeras filas del chunk
    print(chunk.head())

    # Guardar resultados intermedios en una lista
    results.append(chunk)

    # Liberar memoria si no necesitas almacenar los chunks completos
    del chunk

# Combinar todos los chunks procesados en un solo DataFrame final
final_df = pd.concat(results, ignore_index=True)



# Guardar el DataFrame final en un archivo CSV
intermediate_file_path = '/content/drive/My Drive/set_de_datos_Yelp/final_dataframe.csv'
final_df.to_csv(intermediate_file_path, index=False)

# Liberar memoria después de guardar
del final_df
del results
import gc
gc.collect()

# Ahora puedes cargar 'final_dataframe.csv' en cualquier momento para operar sobre él
loaded_df = pd.read_csv(intermediate_file_path)

# Ejemplo de operaciones sobre el DataFrame cargado
print(loaded_df.head())

In [ ]:
#lectura del archivo , en formato csv
intermediate_file_path = '/content/drive/My Drive/set_de_datos_Yelp/final_dataframe.csv'
loaded_df = pd.read_csv(intermediate_file_path)

# Ahora puedes cargar 'final_dataframe.csv' en cualquier momento para operar sobre él
ruta = '/content/drive/My Drive/set_de_datos_Yelp/business_california.xlsx'
business_df = pd.read_excel(ruta)

filtered_reviews = loaded_df[loaded_df['business_id'].isin(business_df['business_id'])]

### transformación

In [ ]:
# Calcular valores nulos y porcentajes
valores_nulos = filtered_reviews.isnull().sum()


# Contar las ocurrencias de cada fila duplicada
duplicate_counts = filtered_reviews[filtered_reviews.duplicated()].sum()



In [1]:


# Escribir el DataFrame como un archivo Parquet
df.to_parquet('Archivos/review_ca.parquet', index=False)


# USER - YELP
## Carga de datos

In [5]:

# lectura del archivo user,parquet
df = pd.read_parquet('D:/usuarios/CIN/MIS DOCUMENTOS/Proyecto_Final/set_de_datos_Yelp/user.parquet')
# lectura del archivo review_ca.parquet para filtrar solo los de CA y disminuir el tamaño del archivo
reviews_ca = pd.read_parquet('Archivos/review_ca.parquet')
#Filtrar las reseñas solo para los user_ID específicos del archivo csv
filtered_reviews_ca = df[df['user_id'].isin(reviews_ca['user_id'])]

### Transformación

In [9]:
# Calcular valores nulos y porcentajes
valores_nulos = filtered_reviews_ca.isnull().sum()
# Contar las ocurrencias de cada fila duplicada
duplicate_counts = filtered_reviews_ca[filtered_reviews_ca.duplicated()].sum()

In [10]:
# Escribir el DataFrame como un archivo Parquet
filtered_reviews_ca.to_parquet('Archivos/user_ca.parquet', index=False)